In [8]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://www.lianjia.com"
html = urlopen(url).read()
bsobj = BeautifulSoup(html, "html5lib")

city_tags = bsobj.find("div", {"class":"fc-main clear"}).findChildren("a")

with open("./citys.csv", "w") as f:
    writ = csv.writer(f)
    for city_tag in city_tags:
        city_url = city_tag.get("href")
        city_name = city_tag.get_text()
        writ.writerow((city_name, city_url))
        print (city_name, city_url)

北京 https://bj.lianjia.com/
保亭 http://you.lianjia.com/bt
保定 https://bd.fang.lianjia.com/
成都 https://cd.lianjia.com/
重庆 https://cq.lianjia.com/
长沙 https://cs.lianjia.com/
澄迈 http://you.lianjia.com/cm
承德 https://chengde.fang.lianjia.com/
滁州 https://cz.fang.lianjia.com/
大连 https://dl.lianjia.com/
东莞 https://dg.lianjia.com/
儋州 http://you.lianjia.com/dz
定安 http://you.lianjia.com/da
大理 http://you.lianjia.com/dl1
德阳 https://dy.fang.lianjia.com/
佛山 https://fs.lianjia.com/
广州 https://gz.lianjia.com/
杭州 https://hz.lianjia.com/
惠州 https://hui.lianjia.com/
海口 https://hk.lianjia.com/
合肥 https://hf.lianjia.com/
衡水 https://hs.fang.lianjia.com/
黄冈 https://hg.fang.lianjia.com/
邯郸 https://hd.fang.lianjia.com/
济南 https://jn.lianjia.com/
嘉兴 https://jx.fang.lianjia.com/
晋中 https://jz.fang.lianjia.com/
昆明 https://km.fang.lianjia.com/
陵水 http://you.lianjia.com/ls
廊坊 https://lf.lianjia.com/
临高 http://you.lianjia.com/lg
乐东 http://you.lianjia.com/ld
龙岩 https://ly.fang.lianjia.com/
乐山 https://leshan.fang.lianjia.

In [36]:
import sys
import csv
def get_city_dict():
    city_dict = {}
    with open("citys.csv", "r") as f:
        reader = csv.reader(f)
        for city in reader:
            #print(city)
            city_dict[city[0]] = city[1]
    return city_dict

city_dict = get_city_dict()
for city_name in city_dict.keys():
    print (city_name)
    
input_city_name = input("Please input city name:")
city_url = city_dict.get(input_city_name)
if city_url:
    print(input_city_name, city_url)
else:
    print("input error")


['北京', 'https://bj.lianjia.com/']
['保亭', 'http://you.lianjia.com/bt']
['保定', 'https://bd.fang.lianjia.com/']
['成都', 'https://cd.lianjia.com/']
['重庆', 'https://cq.lianjia.com/']
['长沙', 'https://cs.lianjia.com/']
['澄迈', 'http://you.lianjia.com/cm']
['承德', 'https://chengde.fang.lianjia.com/']
['滁州', 'https://cz.fang.lianjia.com/']
['大连', 'https://dl.lianjia.com/']
['东莞', 'https://dg.lianjia.com/']
['儋州', 'http://you.lianjia.com/dz']
['定安', 'http://you.lianjia.com/da']
['大理', 'http://you.lianjia.com/dl1']
['德阳', 'https://dy.fang.lianjia.com/']
['佛山', 'https://fs.lianjia.com/']
['广州', 'https://gz.lianjia.com/']
['杭州', 'https://hz.lianjia.com/']
['惠州', 'https://hui.lianjia.com/']
['海口', 'https://hk.lianjia.com/']
['合肥', 'https://hf.lianjia.com/']
['衡水', 'https://hs.fang.lianjia.com/']
['黄冈', 'https://hg.fang.lianjia.com/']
['邯郸', 'https://hd.fang.lianjia.com/']
['济南', 'https://jn.lianjia.com/']
['嘉兴', 'https://jx.fang.lianjia.com/']
['晋中', 'https://jz.fang.lianjia.com/']
['昆明', 'https://km.f

In [ ]:
def get_district_dict(url):
    district_dict = {}
    html = urlopen(url).read()
    bsobj = BeautifulSoup(html, "html5lib")
    roles = bsobj.find("div", {"data-role":"ershoufang"}).findChildren("a")
    for role in roles:
        district_url = role.get("href")
        district_name = role.get_text()
        district_dict[district_name] = district_url
    return district_dict
def run():
    city_dict = get_city_dict()
    for city in city_dict.keys():
        print(city)
    input_city = input("Please input city name:")
    city_url = city_dict.get(input_city)
    if not city_url:
        print("input error")
        sys.exit()
    ershoufang_city_url = city_url + "ershoufang"
    district_dict = get_district_dict(ershoufang_city_url)
    for district in district_dict.keys():
        print(district)
    input_district = input("Please input area:")
    district_url = district_dict.get(input_district)
    if not district_url:
        print("input error")
        sys.exit()
    house_info_url = city_url + district_url[1:]
    print(house_info_url)
    house(house_info_url)
if __name__ == "__main__":
    run()

In [ ]:
# -*- coding: utf-8 -*-
# filename: house_info.py

import sys
import re
import csv
import urllib

from bs4 import BeautifulSoup


# 成功打开页面时返回页面对象，否则打印错误信息，退出程序
def get_bsobj(url):
    page = urllib.request.urlopen(url)
    # 状态码为 200 时页面成功打开
    if page.getcode() == 200:
        html = page.read()
        bsobj = BeautifulSoup(html, "html5lib")
        return bsobj
    else:
        print ("页面错误")
        sys.exit()

# 将页面中每一条房屋信息保存为一个字典，将所有的字典保存在列表中，返回列表
def get_house_info_list(url):
    house_info_list = []
    bsobj = get_bsobj(url)
    if not bsobj:
        return None

    house_list = bsobj.find_all("li", {"class":"clear"})

    for house in house_list:
        # 标题
        title = house.find("div", {"class": "title"}).get_text().encode("utf-8")

        # 获取信息数据（例：加怡名城 | 2室1厅 | 62.48平米 | 西 | 精装），通过“|”符号分割字符串
        info = house.find("div", {"class": "houseInfo"}).get_text().split("|")

        # 小区（例：加怡名城），strip()去除字符串两边的空格，encode，将字符串编码成 utf-8 格式
        block = info[0].strip()

        # 房型（例：2室一厅）
        house_type = info[1].strip()

        # 面积大小，保留整数（例：62.48平米，保留整数后为 62）
        size_info = info[2].strip()
        size = re.findall(r"\d+", size_info)[0]

        # 价格，保留整数（例：120.3万，保留整数后为 120）
        price_info = house.find("div", {"class": "totalPrice"}).span.get_text()
        price = re.findall(r"\d+", price_info)[0]

        # 添加到列表中
        house_info_list.append({
            "title": title,
            "price": int(price),
            "size": int(size),
            "block": block,
            "house_type": house_type
        })
    return house_info_list


# 读取前三个页面的房屋信息，将信息保存到 house.csv 文件中
def house(url):
    house_info_list = []

    # range(3),即前三个子页面
    for i in range(3):
        new_url = url + 'pg' + str(i+1)
        house_info_list.extend(get_house_info_list(new_url))

    if house_info_list:
        # 将数据保存到 house.csv 文件中
        with open("./house.csv", "wb+") as f:
            # writer 对象，修改默认分隔符为 "|"
            writer = csv.writer(f, delimiter="|")
            for house_info in house_info_list:
                title = house_info.get("title")
                price = house_info.get("price")
                size = house_info.get("size")
                block = house_info.get("block")
                house_type = house_info.get("house_type")
                # 写入一行
                writer.writerow([title, int(price), int(size), block, house_type])
                print (block, price, size)

